In [15]:
#I am trying to scrap the customer reviews of Amazon to analyse the market by sentiment analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [2]:
no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            #print(price.text)
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [3]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [4]:
df2 = pd.read_csv("amazon_products.csv")
df2

,Book Name,Author,Rating,Customers_Rated,Price
0,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5 out of 5 stars,"9,459",₹ 399.00
1,World’s Greatest Books For Personal Growth & W...,Dale Carnegie,4.5 out of 5 stars,"1,720",₹ 299.00
2,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6 out of 5 stars,"6,210",₹ 362.00
3,One Arranged Murder,Chetan Bhagat,4.2 out of 5 stars,"3,154",₹ 143.00
4,Think Like a Monk,Jay Shetty,4.8 out of 5 stars,"4,377",₹ 367.00
...,...,...,...,...,...
95,Deep Work: Rules for Focused Success in a Dist...,Cal Newport,4.4 out of 5 stars,"3,856",₹ 270.00
96,CBSE All In One English Core Class 12 for 2021...,Gajendra Singh,4.3 out of 5 stars,204,₹ 402.00
97,Peppa Pig: Little Library,Peppa Pig,4.3 out of 5 stars,"8,957",₹ 189.00
98,Think & Grow Rich (Hindi),Napoleon Hill,4.3 out of 5 stars,"1,667",₹ 127.00


In [5]:
df2.shape

(100, 5)

In [61]:
def clean_text(Name):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', Name).split())

In [24]:
df2['Rating_Score'] = df2['Rating'].str.extract('(\d+.\d+)').astype(float)

In [25]:
df2

,Book Name,Author,Rating,Customers_Rated,Price,Rating_Score
0,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5 out of 5 stars,"9,459",₹ 399.00,4.5
1,World’s Greatest Books For Personal Growth & W...,Dale Carnegie,4.5 out of 5 stars,"1,720",₹ 299.00,4.5
2,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6 out of 5 stars,"6,210",₹ 362.00,4.6
3,One Arranged Murder,Chetan Bhagat,4.2 out of 5 stars,"3,154",₹ 143.00,4.2
4,Think Like a Monk,Jay Shetty,4.8 out of 5 stars,"4,377",₹ 367.00,4.8
...,...,...,...,...,...,...
95,Deep Work: Rules for Focused Success in a Dist...,Cal Newport,4.4 out of 5 stars,"3,856",₹ 270.00,4.4
96,CBSE All In One English Core Class 12 for 2021...,Gajendra Singh,4.3 out of 5 stars,204,₹ 402.00,4.3
97,Peppa Pig: Little Library,Peppa Pig,4.3 out of 5 stars,"8,957",₹ 189.00,4.3
98,Think & Grow Rich (Hindi),Napoleon Hill,4.3 out of 5 stars,"1,667",₹ 127.00,4.3


In [73]:
def analyze_sentiment(Rating_Score):
    if Rating_Score > 4.5:
        return 'Best Choice'
    elif Rating_Score == 4.0:
        return 'Better Choice'
    else:
        return 'Good Choice'

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Book Name        100 non-null    object
 1   Author           100 non-null    object
 2   Rating           100 non-null    object
 3   Customers_Rated  100 non-null    object
 4   Price            100 non-null    object
 5   Rating_Score     100 non-null    int32 
 6   Comments         100 non-null    object
dtypes: int32(1), object(6)
memory usage: 5.2+ KB


In [75]:
df2['Comments'] = df2['Rating_Score'].apply(lambda x: analyze_sentiment(x))
df2.head()

,Book Name,Author,Rating,Customers_Rated,Price,Rating_Score,Comments
0,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5 out of 5 stars,"9,459",₹ 399.00,4.5,Good Choice
1,World’s Greatest Books For Personal Growth & W...,Dale Carnegie,4.5 out of 5 stars,"1,720",₹ 299.00,4.5,Good Choice
2,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6 out of 5 stars,"6,210",₹ 362.00,4.6,Best Choice
3,One Arranged Murder,Chetan Bhagat,4.2 out of 5 stars,"3,154",₹ 143.00,4.2,Good Choice
4,Think Like a Monk,Jay Shetty,4.8 out of 5 stars,"4,377",₹ 367.00,4.8,Best Choice


In [47]:
#Web Scraping
#import the python request library to query a website
import requests
#specify the url we want to scrape from
Link = "https://www.consumeraffairs.com/online/amazon.html?page=3#sort=top_reviews&filter=none"
#convert the web page to text
Link_text = requests.get(Link).text
print(Link_text)

<!doctype html><html lang="en" class="no-js" ><head><!-- Google Tag Manager --><script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.defer=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-WSBZRR');</script><!-- End Google Tag Manager --><meta http-equiv="X-UA-Compatible" content="IE=edge"><link rel="manifest" href="//media.consumeraffairs.com/static/manifest.9cfa8161eca4.json"><meta charset="utf-8"/><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"UgUFVVZVGwIBXVFRBQQG",licenseKey:"e2e3a3da7b",applicationID:"11903251"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"

In [48]:
#import BautifulSoup library to pull data out of HTML and XML files
from bs4 import BeautifulSoup
#to convert Link_text into a BeautifulSoup Object
soup = BeautifulSoup(Link_text, 'lxml')
print(soup)

<!DOCTYPE html>
<html class="no-js" lang="en"><head><!-- Google Tag Manager --><script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.defer=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-WSBZRR');</script><!-- End Google Tag Manager --><meta content="IE=edge" http-equiv="X-UA-Compatible"/><link href="//media.consumeraffairs.com/static/manifest.9cfa8161eca4.json" rel="manifest"/><meta charset="utf-8"/><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"UgUFVVZVGwIBXVFRBQQG",licenseKey:"e2e3a3da7b",applicationID:"11903251"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("functio

In [49]:
#make the indentation proper
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" lang="en">
 <head>
  <!-- Google Tag Manager -->
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.defer=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-WSBZRR');
  </script>
  <!-- End Google Tag Manager -->
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <link href="//media.consumeraffairs.com/static/manifest.9cfa8161eca4.json" rel="manifest"/>
  <meta charset="utf-8"/>
  <script type="text/javascript">
   (window.NREUM||(NREUM={})).loader_config={xpid:"UgUFVVZVGwIBXVFRBQQG",licenseKey:"e2e3a3da7b",applicationID:"11903251"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.export

In [50]:
#To take a look at the title of the web page
print(soup.title)

<title>Top 2,496 Amazon.com Reviews | Page 3</title>


In [51]:
#Only the string not the tags
print(soup.title.string)

Top 2,496 Amazon.com Reviews | Page 3


In [52]:
#First <a></a> tag
soup.a

<a aria-label="ConsumerAffairs" class="ca-hdr__logo" data-uapi-link="header_logo" href="https://www.consumeraffairs.com/"><svg viewbox="0 0 443 44" xmlns="http://www.w3.org/2000/svg"><path class="ca-icon__colored-fill" d="M329.789 41.882l1.395-.349c1.906-.448 2.074-.729 2.074-2.915V15.865h-3.7v-.448l3.7-1.569v-2.131c0-6.728 4.148-11.1 10.876-11.1 3.531 0 5.55 1.514 5.55 3.42A2.357 2.357 0 01347.1 6.5c-1.738 0-2.522-.784-2.915-3.083-.168-1.121-.392-1.738-1.289-1.738-2.13 0-3.812 2.13-3.812 7.568V13.9h9.025l.004-2.744C348.113 4.261 353.047 0 359.718 0c3.588 0 5.663 1.458 5.663 3.532a2.52 2.52 0 01-2.747 2.579c-1.738 0-2.524-.785-2.972-3.2-.224-1.177-.392-1.794-1.346-1.794-2.242 0-4.372 2.074-4.372 7.513v5.27h5.83l-.224 1.962h-5.606v22.821c0 2.017.168 2.354 1.738 2.69l2.523.561v.56h-13.623v-.56l1.518-.334c1.849-.45 2.017-.73 2.017-2.915v-22.82h-9.025v22.761c0 2.131.168 2.467 1.738 2.8l2.355.5v.56h-13.396v.008h-14.861v-.616l1.29-.28c1.849-.45 2.242-1.178 2.242-2.3a12.225 12.225 0 00-.729-3

In [53]:
#all the <a> </a> tags
soup.find_all('a')

[<a aria-label="ConsumerAffairs" class="ca-hdr__logo" data-uapi-link="header_logo" href="https://www.consumeraffairs.com/"><svg viewbox="0 0 443 44" xmlns="http://www.w3.org/2000/svg"><path class="ca-icon__colored-fill" d="M329.789 41.882l1.395-.349c1.906-.448 2.074-.729 2.074-2.915V15.865h-3.7v-.448l3.7-1.569v-2.131c0-6.728 4.148-11.1 10.876-11.1 3.531 0 5.55 1.514 5.55 3.42A2.357 2.357 0 01347.1 6.5c-1.738 0-2.522-.784-2.915-3.083-.168-1.121-.392-1.738-1.289-1.738-2.13 0-3.812 2.13-3.812 7.568V13.9h9.025l.004-2.744C348.113 4.261 353.047 0 359.718 0c3.588 0 5.663 1.458 5.663 3.532a2.52 2.52 0 01-2.747 2.579c-1.738 0-2.524-.785-2.972-3.2-.224-1.177-.392-1.794-1.346-1.794-2.242 0-4.372 2.074-4.372 7.513v5.27h5.83l-.224 1.962h-5.606v22.821c0 2.017.168 2.354 1.738 2.69l2.523.561v.56h-13.623v-.56l1.518-.334c1.849-.45 2.017-.73 2.017-2.915v-22.82h-9.025v22.761c0 2.131.168 2.467 1.738 2.8l2.355.5v.56h-13.396v.008h-14.861v-.616l1.29-.28c1.849-.45 2.242-1.178 2.242-2.3a12.225 12.225 0 00-.729-

In [55]:
#Fetch all the table tags
all_table = soup.find('')
print(all_table)

<html class="no-js" lang="en"><head><!-- Google Tag Manager --><script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.defer=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-WSBZRR');</script><!-- End Google Tag Manager --><meta content="IE=edge" http-equiv="X-UA-Compatible"/><link href="//media.consumeraffairs.com/static/manifest.9cfa8161eca4.json" rel="manifest"/><meta charset="utf-8"/><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"UgUFVVZVGwIBXVFRBQQG",licenseKey:"e2e3a3da7b",applicationID:"11903251"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_